In [12]:
import glob, os
%matplotlib inline

import numpy as np
import IPython
from IPython.display import HTML
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

plt.rcParams.update({'font.size': 14})

from joblib import load, dump

In [13]:
pathSave = "Data_Files/"

In [ ]:
rho_m  = 3500. 
g     = 3.7 
alpha_m = 2.5e-5
T_delta = 2000. 
D = 1700e+3         
k_diffusive = 1e-6 
R = 8.314   

def format_func(_val, tick_number):
    f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
    _g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = mticker.FuncFormatter(_g)
    return "{}".format(fmt(_val))

_dict = {} 
dict_ra = "dictResults_indGaussians[[12, 6], 3, [0, 1, 2, 3, 4, 5], [0, 100], 0.8, 0, 'gaussian', 'tanh', 0.0].txt"
with open(pathSave +  dict_ra, "rb") as myFile:
    _dict["para0"] = load(myFile)  
dict_E = "dictResults_indGaussians[[12, 6], 3, [0, 1, 2, 3, 4, 5], [0, 100], 0.8, 4, 'gaussian', 'tanh', 0.0].txt"
with open(pathSave +  dict_E, "rb") as myFile:
    _dict["para1"] = load(myFile)  
dict_V = "dictResults_indGaussians[[12, 6], 3, [0, 1, 2, 3, 4, 5], [0, 100], 0.8, 5, 'gaussian', 'tanh', 0.0].txt"
with open(pathSave +  dict_V, "rb") as myFile:
    _dict["para2"] = load(myFile) 
dict_L = "dictResults_indGaussians[[12, 6], 3, [0, 1, 2, 3, 4, 5], [0, 100], 0.8, 6, 'gaussian', 'tanh', 0.0].txt"
with open(pathSave +  dict_L, "rb") as myFile:
    _dict["para3"] = load(myFile) 
dict_T = "dictResults_indGaussians[[12, 6], 3, [0, 1, 2, 3, 4, 5], [0, 100], 0.8, 7, 'gaussian', 'tanh', 0.0].txt"
with open(pathSave +  dict_T, "rb") as myFile:
    _dict["para4"] = load(myFile) 
    
paraVecTxt = ["Reference viscosity", "Activation energy", "Activation volume", "Enrichment factor", "Initial temperature"]
paraVec = ['Ra', 'ERef', 'VRef', 'Enrichment_cr', 'iniTempTop']
paraVecFull = [r"$\log(\eta_{ref})$ [Pa s]", r"$E$ [J mol$^{-1}$]", r"$V$ [m$^3$ mol$^{-1}$]", "$\Lambda$", "$T_{ini}$ [K]"]
colors = ['r', 'g', 'm']
fig = plt.figure(figsize=(14,14), dpi=320) 
nRows = 6
nCols = 5
plotCounter = 1
for rowInd in range(nRows):
    for paraInd in range(nCols):
        ax = fig.add_subplot(nRows,nCols,plotCounter)
        plotCounter += 1
        if rowInd==0:
            ax.plot(-np.asarray(_dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + "_loss"]), 'r-', label='Training')
            ax.plot(-np.asarray(_dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + "_loss_cv"]),"b--", label='Validation')
            ax.set_xlabel('Epochs')
            if paraInd==0:
                ax.set_ylabel('Log-likelihood')
            if paraInd==2:
                ax.legend()
            ax.locator_params(axis='x', nbins=2)
            ax.set_yticks([-1.5,-0.5,0.5,0.5,1.5,2.5])
            ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f'))
            ax.set_title(paraVecTxt[paraInd],fontweight='bold',fontsize=13)
        elif rowInd==1: 
            x = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Testx"]
            y = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Testy"]
            Pr_xx = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_Pr_xx"]
            ax.contourf(x,y,Pr_xx)
            ax.set_xlabel("True")
            if paraInd==0:
                ax.set_ylabel("Predicted")
            if paraInd==0:
                ax.set_xticks([19,20,21,22])
                ax.set_yticks([19,20,21,22])
            elif paraInd==1:
                ax.set_xticks([1e+5,3e+5,5e+5])
                ax.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
                for label in ax.get_xticklabels():
                    label.set_horizontalalignment('right')
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            elif paraInd==2:
                ax.set_xticks([4e-6, 7e-6, 10e-6])
                ax.set_yticks([4e-6, 6e-6, 8e-6, 10e-6])
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            elif paraInd==3:
                ax.set_xticks([1,10,20,30,40,50])
                ax.set_yticks([1,10,20,30,40,50])
            elif paraInd==4:
                ax.set_xticks([1600,1700,1800])
                ax.set_yticks([1600,1650,1700,1750,1800])
        else: 
            if paraInd==0:
                ax.set_ylabel(r"$p(\mathbf{p}| \mathbf{o})$")
            if rowInd==2:
                indActual = 60
            elif rowInd==3:
                indActual = 180
            elif rowInd==4:
                indActual = 360
            elif rowInd==5:
                indActual = 540
            xxSorted = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_xxSorted"]
            val = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_trueVal_" + str(indActual)]
            yAggregator = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_pdf_mixture" + str(0) + "_case_" + str(indActual)] + \
                    _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_pdf_mixture" + str(1) + "_case_" + str(indActual)] + \
                    _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_pdf_mixture" + str(2) + "_case_" + str(indActual)] 
            for i in range(3):
                yPDF = _dict["para"+str(paraInd)]["forFig_" + paraVec[paraInd] + ", Test_pdf_mixture" + str(i) + "_case_" + str(indActual)]
                ax.plot(xxSorted,yPDF,colors[i]+'--', linewidth=2)
            ax.plot(xxSorted, yAggregator,"k-", linewidth=2)
            ax.plot([val, val],[0,max(yAggregator)], "-", color='grey', linewidth=1.5)
            if paraInd==1:
                ax.set_xticks([1e+5,3e+5,5e+5])
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                for label in ax.get_xticklabels():
                    label.set_horizontalalignment('right')
            elif paraInd==0:
                ax.set_xticks([19,20,21,22])
            elif paraInd==3:
                ax.set_xticks([1,10,20,30,40,50])
            elif paraInd==4:
                ax.set_xticks([1600,1700,1800])
            elif paraInd==2:
                ax.set_xticks([4e-6, 7e-6, 10e-6])
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            if rowInd==5:
                ax.set_xlabel(paraVecFull[paraInd])
plt.tight_layout(pad=0.3)
#plt.savefig(pathSave + 'Diagrams_dimDistributions.pdf', bbox_inches='tight')

In [ ]:
_dict = {}
paraVecInd = [0, 4, 5, 6, 7]
obserVec = [[1], \
            [1,2], \
            [1, 2, 5], \
            [0,1,2,3,4,5]]

for indp, p in enumerate(paraVecInd):
    for indo, o in enumerate(obserVec):  
        dict_toOpen = 'dictResults[[12, 6], 3, ' + str(o) + ', [0, 100], 0.8, ' + str(p) + """, 'gaussian', 'tanh', 0.0].txt"""
        with open(pathSave +  dict_toOpen, "rb") as myFile:
            _dict[str(indp)+str(indo)] = load(myFile) 


fig = plt.figure(figsize=(14,12), dpi=320) 
nRows = 5
nCols = 5
plotCounter = 1
for rowInd in range(nRows):
    for colInd in range(nCols):
        ax = fig.add_subplot(nRows,nCols,plotCounter)
        plotCounter += 1
        if rowInd==4:
            stdVec = []
            logVec = []
            for i in range(4):
                logVec.append(_dict[str(colInd)+str(i)]["forFig_" + paraVec[colInd] + "_loss_cv"][-1])
                stdVec.append(_dict[str(colInd)+str(i)]["forFig_" + paraVec[colInd] + ", Test_averageSTD"])
            if colInd==0:
                rangePara = 22-19
            elif colInd==1:
                rangePara = 5e+5-1e+5
            elif colInd==2:
                rangePara = 1e-5-4e-6
            elif colInd==3:
                rangePara=50-1
            else:
                rangePara=1800-1600
            if colInd==2:
                stdplot = np.mean(4*np.asarray(stdVec)/rangePara) + np.zeros((len(stdVec),1))
            else:
                stdplot = 4*np.asarray(stdVec)/rangePara
            ax.plot(-np.asarray(logVec),stdplot,linewidth=5)
            ax.set_xlabel("Log-likelihood")
            #ax.set_ylim([0.0,1.2])
            if colInd==0:
                ax.set_ylabel("$\pm 2 \overline{\sigma}/ \; range$")
            if colInd==2:
                ax.set_xticks([(-np.asarray(logVec)).min(),(-np.asarray(logVec)).max()])
        else:
            x = _dict[str(colInd)+str(rowInd)]["forFig_" + paraVec[colInd] + ", Testx"]
            y = _dict[str(colInd)+str(rowInd)]["forFig_" + paraVec[colInd] + ", Testy"]
            Pr_xx = _dict[str(colInd)+str(rowInd)]["forFig_" + paraVec[colInd] + ", Test_Pr_xx"]
            ax.contourf(x,y,Pr_xx)
            ax.set_xlabel("True")
            if colInd==0:
                ax.set_ylabel("Predicted")
            if colInd==0:
                ax.set_xticks([19,20,21,22])
                ax.set_yticks([19,20,21,22])
            elif colInd==1:
                ax.set_xticks([1e+5,3e+5,5e+5])
                ax.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
                for label in ax.get_xticklabels():
                    label.set_horizontalalignment('right')
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            elif colInd==2:
                ax.set_xticks([4e-6, 7e-6, 10e-6])
                ax.set_yticks([4e-6, 6e-6, 8e-6, 10e-6])
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            elif colInd==3:
                ax.set_xticks([1,10,20,30,40,50])
                ax.set_yticks([1,10,20,30,40,50])
            elif colInd==4:
                ax.set_xticks([1600,1700,1800])
                ax.set_yticks([1600,1650,1700,1750,1800])
            if rowInd==0:
                ax.set_title(paraVecFull[colInd])
            sigVal = _dict[str(colInd)+str(rowInd)]["forFig_" + paraVec[colInd] + ", Test_averageSTD"]
            if colInd==1 or colInd==2:
                sigVal = str(format_func(sigVal, None))
            else:
                sigVal = str(round(sigVal,2))
            ax.text(0.95, 0.01, "$\overline{\sigma}=$"+sigVal,
                    verticalalignment='bottom', horizontalalignment='right',
                    transform=ax.transAxes,
                    color='white', fontsize=15)
plt.tight_layout(pad=0.2)
plt.show()

In [ ]:
_dict = {}
paraVecInd = [0, 4, 5, 6, 7]
obserVec = [0.1,0.5,1]

for indo, o in enumerate(obserVec):  
    dict_toOpen = "multiVar_noise_dict" + str(o) + ".txt"
    with open(pathSave +  dict_toOpen, "rb") as myFile:
        _dict[str(indo)] = load(myFile) 

fig = plt.figure(figsize=(14,8), dpi=320) 
nRows = 3
nCols = 5
plotCounter = 1
for rowInd in range(nRows):
    for colInd in range(nCols):
        ax = fig.add_subplot(nRows,nCols,plotCounter)
        plotCounter += 1
        x = _dict[str(rowInd)]["forFig_" + paraVec[colInd] + ", Testx"]
        y = _dict[str(rowInd)]["forFig_" + paraVec[colInd] + ", Testy"]
        Pr_xx = _dict[str(rowInd)]["forFig_" + paraVec[colInd] + ", Test_Pr_xx"]
        ax.contourf(x,y,Pr_xx)
        ax.set_xlabel("True")
        if colInd==0:
            ax.set_ylabel("Predicted")
        if colInd==0:
            ax.set_xticks([19,20,21,22])
            ax.set_yticks([19,20,21,22])
        elif colInd==1:
            ax.set_xticks([1e+5,3e+5,5e+5])
            ax.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
            for label in ax.get_xticklabels():
                label.set_horizontalalignment('right')
            ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
        elif colInd==2:
            ax.set_xticks([4e-6, 7e-6, 10e-6])
            ax.set_yticks([4e-6, 6e-6, 8e-6, 10e-6])
            ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
        elif colInd==3:
            ax.set_xticks([1,10,20,30,40,50])
            ax.set_yticks([1,10,20,30,40,50])
        elif colInd==4:
            ax.set_xticks([1600,1700,1800])
            ax.set_yticks([1600,1650,1700,1750,1800])
        if rowInd==0:
            ax.set_title(paraVecFull[colInd])
        sigVal = _dict[str(rowInd)]["forFig_" + paraVec[colInd] + ", Test_averageSTD"]
        if colInd==1 or colInd==2:
            sigVal = str(format_func(sigVal, None))
        else:
            sigVal = str(round(sigVal,2))
        ax.text(0.95, 0.01, "$\overline{\sigma}=$"+sigVal,
                verticalalignment='bottom', horizontalalignment='right',
                transform=ax.transAxes,
                color='white', fontsize=16)
plt.tight_layout(pad=0.2)
plt.show()